### This notebook is an advanced tutorial detailing the config changes for optimising the BERT and LSTM models for Experiencer classification task on custom dataset

In [1]:
import json
import os
from datetime import date
from medcat.cat import CAT
from medcat.meta_cat import MetaCAT
from medcat.config_meta_cat import ConfigMetaCAT
from medcat.tokenizers.meta_cat_tokenizers import TokenizerWrapperBPE, TokenizerWrapperBERT
from tokenizers import ByteLevelBPETokenizer

In [2]:
# if you want to enable info level logging
import logging
logging.basicConfig(level=logging.INFO,force=True)
logger = logging.getLogger(__name__)

# Set parameters

In [3]:
# relative path to working_with_cogstack folder
_rel_path = os.path.join("..", "..", "..")
# absolute path to working_with_cogstack folder
base_path = os.path.abspath(_rel_path)
# Load mct export
ann_dir = os.path.join(base_path, "data", "medcattrainer_export")

mctrainer_export_path = ann_dir + ""  # name of your mct export

# Load model
model_dir = os.path.join(base_path, "models", "modelpack")
modelpack = '' # name of modelpack
model_pack_path = os.path.join(model_dir, modelpack)
     #output_modelpack = model_dir + f"{today}_trained_model"

# will be used to date the trained model
today = str(date.today())
today = today.replace("-","")

# Initialise meta_ann models
if model_pack_path[-4:] == '.zip':
    base_dir_meta_models = model_pack_path[:-4]
else:
    base_dir_meta_models = model_pack_path

# Iterate through the meta_models contained in the model
meta_model_names = [] # These Meta_annotation tasks should correspond to the ones labelled in the mcttrainer export
for dirpath, dirnames, filenames in os.walk(base_dir_meta_models):
    for dirname in dirnames:
        if dirname.startswith('meta_'):
            meta_model_names.append(dirname[5:])

Before you run the next section please double check that the model meta_annotation names matches to those specified in the mct export.



# Class weights 

Adjusting class weights to give more importance to specific classes. Generally, class weights are used in favour of minority classes(classes with less number of samples) to boost their performance.
<br><br>To use class weights, we have 2 options:
<br>1. calculate class weights based on class distribution
<br>2. using specified class weights


#option 1 <br>
mc.config.train['class_weights'] = []<br>
mc.config.train['compute_class_weights'] = True<br>
#NOTE: this will only be applicable if mc.config.train.class_weights is empty<br>
<br>
#2nd option<br>
#using specified class weights<br>
mc.config.train['class_weights'] = [0.4,0.3,0.1]<br>

NOTE: Make sure to correctly map the class weights to their corresponding class index (ID). <br>To check the index assigned to the classes, use: <br>`print(mc.config.general['category_value2id'])`
<br>This will print a dictionary where the class names and their corresponding IDs (indices) are displayed. <br>
The first position in the class weight list corresponds to the class with ID 0 in the dictionary, and so on.

# 2 phase learning for training

2 phase learning is used to mitigate class imbalance. In 2 phase learning, the models are trained twice: <br> 
Phase 1: trains for minority class(es) by undersampling data so that there is no class imbalance
<br>Phase 2: trains for all classes

Phase 1 ensures that the model learns minority class(es) and captures the details correctly.
<br> Phase 2 is when the model is expected to learn the majority class as it is trained on the entire dataset.

Paper reference - https://ieeexplore.ieee.org/document/7533053
<br>NOTE: Make sure to use class weights in favour of minority classes with 2 phase learning

In [ ]:
#--------------------------------Phase 1--------------------------------
def run_phase_1(meta_model,class_wt_phase1 = None):
    #Loading the pre-defined config for phase 1
    config_ph_1_path = os.path.join(base_dir_meta_models,"meta_"+meta_model,"config_ph1.json")
    with open(config_ph_1_path) as f:
        config_ph1 = json.load(f)

    mc = MetaCAT.load(save_dir_path=os.path.join(base_dir_meta_models,"meta_"+meta_model),config_dict = config_ph1)

    if class_wt_phase1:
        mc.config.train['class_weights'] = class_wt_phase1

    mc.config.train['nepochs'] = 30 #You can change the number of epochs, remember to keep them higher for phase 1

    save_dir_path= "test_meta_"+meta_model # Where to save the meta_model and results. 
    results = mc.train_from_json(mctrainer_export_path, save_dir_path=save_dir_path)
    # Save results
    json.dump(results['report'], open(os.path.join(save_dir_path,'meta_'+meta_model+'_results_phase1.json'), 'w'))

#--------------------------------Phase 2--------------------------------
def run_phase_2(meta_model,class_wt_phase2 = None): 
    #Loading the pre-defined config for phase 2
    config_ph_2_path = os.path.join(base_dir_meta_models,"meta_"+meta_model,"config_ph2.json")
    with open(config_ph_2_path) as f:
        config_ph2 = json.load(f)

    mc = MetaCAT.load(save_dir_path=os.path.join(base_dir_meta_models,"meta_"+meta_model),config_dict = config_ph2)

    if class_wt_phase2:
        mc.config.train['class_weights'] = class_wt_phase2

    mc.config.train['nepochs'] = 15

    save_dir_path= "test_meta_"+meta_model # Where to save the meta_model and results. Ensure to keep this same as Phase 1
    results = mc.train_from_json(mctrainer_export_path, save_dir_path=save_dir_path)
    # Save results
    json.dump(results['report'], open(os.path.join(save_dir_path,'meta_'+meta_model+'_results_phase2.json'), 'w'))

#--------------------------------Driver--------------------------------
for meta_model in meta_model_names:
    #To use your own class weights instead of the pre-defined ones for the 2 phases, uncomment the below lines
    '''class_wt_phase1 = []
    class_wt_phase2 = []'''

    # Train 2 phase learning
    logger.info("\n********************Beginning Phase 1********************")
    run_phase_1(meta_model,class_wt_phase1)
    logger.info("\n********************Beginning Phase 2********************")
    run_phase_2(meta_model,class_wt_phase2)

# Oversampling data

You can generate synthetic data to help mitigate class imbalance. <br> Use this code to generate synthetic data using LLM - [link](https://gist.github.com/shubham-s-agarwal/401ef8bf6cbbd66fa0c76a8fbfc1f6c4) <br> <b>NOTE</b>: the generated data will require manual quality check to ensure that high quality and relevant data is used for training. 

The data generated from the gist code and the format of the data required by MedCAT are different, requiring manual formatting at the moment. We will update this module to include the code to handle the same.

In [ ]:
# To run the training with original + synthetic data
# Follow all the same steps till initializing the metacat model

# Initialise and train meta_model
mc = MetaCAT(tokenizer=tokenizer, embeddings=None, config=config)

# the format expected is [[['text','of','the','document'], [index of medical entity], "label" ],
#                ['text','of','the','document'], [index of medical entity], "label" ]]

synthetic_data_export = [[],[],[]]

results = mc.train_from_json(mctrainer_export_path, save_dir_path=save_dir_path,data_oversampled=synthetic_data_export)

# Save results
json.dump(results['report'], open(os.path.join(save_dir_path,'meta_'+meta_model+'_results.json'), 'w'))